In [ ]:
#%matplotlib inline

In [5]:
#PASO 1
from skimage import io #Librerias para importar las imagenes

#PASO 2
from skimage import img_as_ubyte#Mostrar las 3 imagenes
from skimage.color import rgb2grey#Mostrar las 3 imagenes
from skimage.color import rgb2hsv#Mostrar las 3 imagenes
import numpy as np#Mostrar las 3 imagenes
import matplotlib.pyplot as plt#Mostrar las 3 imagenes

#PASO 3
from skimage.filters import threshold_otsu

#PASO 4

#PASO 5
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image, remove_small_objects
from skimage.morphology import disk, reconstruction
from skimage.morphology import binary_erosion,rectangle,binary_dilation

#PASO 6
from skimage.transform import hough_line, hough_line_peaks,probabilistic_hough_line
from skimage.feature import canny
from skimage import data
##imports para el uso de interactive
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

#PASO 7

#PASO 8
import math

#PASO 9
import networkx as nx
from networkx.algorithms import approximation as apxa

#PASO 1
##Funcion Para leer una imagen a traves de scikit-Image (skimage)
#dandole el path de la imagen
#Return: Img imagen leida
def leerImagen(pathImg):
    #pathImg="/Users/Ismael/Desktop/TFG_DietaPorDientes/TrabajosPasadosPorJose/dietaJose/ATP02 UE4 46.jpg"
    img = io.imread(pathImg)
    return img

#PASO 2
#Entrada: img imagen original leida anteriormente
#Pasamos la imagen al espacio de color RGB y nos quedamos con el canal rojo
#Pasamos la imagen al espacio de color HSV
#Normalizamos la imagen del espacio de color HSV para utilizar que distancia 
#  al rojo tenemos y poder hacer el theshold
#Return: distance_red distancia de cada pixel al rojo apra luego hacer el threshold
def distanciaAlRojo(img):
    rojo=img[:,:,0]
    imgHSV=rgb2hsv(img)
    distance_red = rgb2grey(1 - np.abs(imgHSV - (0, 1, 0)))
    return distance_red

#PASO 3
#Entrada: distance_red distancia de cada pixel al rojo apra luego hacer el threshold
#Binarizamos la imagen para obtener una clara diferenciacion entre 
#el fondo y las lineas que queremos detectar
#Return: imgBin imagen binarizada
def binarizar(distance_red):
    threshold_global_otsu = threshold_otsu(distance_red)
    imgBin = distance_red >= threshold_global_otsu
    return imgBin

#PASO 4
#Entrada:  imgBin imagen binarizada
#Como la imagen no hay qeu procesarla completa porque solo tiene una region
#marcada con lineas tenemos qque recortarla
#Return: 
#    imgBinCrop: Es la imagen binarizada recortada a la region que queremos
#    imgCrop: Es la imagen original recortada a la region que queremos
def cropImg(imgBin,img):
    imgBinCrop = imgBin[0:750,500:1500]
    imgCrop = img[0:750,500:1500]

    #imgBinCrop = imgBin
    #imgCrop = img

    #imgBinCrop = imgBin[0:750,0:750]
    #imgCrop = img[0:750,0:750]
    return imgBinCrop,imgCrop

#PASO 5
#Entrada: imgBinCrop Es la imagen binarizada recortada a la region que queremos
#Esta funcion en lo que consiste es qeu entrando la imagen binarizada 
#reduzcamos el tamaño de las lineas para que la deteccion tenga menos
#incertidumbre y sea mas rápida la deteccion de lineas
#Return: sinRuido Es la imagen una vez corregida la binarizacion apra que las 
#        lineas sean mas delgadas.
def reducirGrosor(imgBinCrop):
    selem2 = rectangle(8,1)
    selem = rectangle(3,1)
    #sinRuido = binary_dilation(imgBinCrop,selem2)
    #sinRuido = binary_erosion(imgBinCrop,selem)
    sinRuido = skeletonize(imgBinCrop)
    return sinRuido

#PASO 6
#Entrada:
#    threshold: Densidad de lineas 
#    line_length: Distancia minima de las lineas que calcula.
#    line_gap: Distancia a la que si estan los extremos de las rectas las una
#    sinRuido:Es la imagen una vez corregida la binarizacion apra que las 
#        lineas sean mas delgadas.
#Esta funcion lo que va a hacer es calcular los segmentos(ORIGEN"Ox,Oy" Y FIN "FX,FY") 
#de cada una de las lineas que se han detectado en la imagen binarizada atraves de la 
#funcion probabilistica de la transformada de hough
#Return: lines Lista de todos los segmentos que ha detectado.
def proHough(threshold,line_length,line_gap,sinRuido):
    lines = probabilistic_hough_line(sinRuido, threshold, line_length, line_gap)
    return lines 

#PASO 7
#------------------Funciones auxiliares para la union de segmentos----------------------
#Entrada:
#        seg1: ((px,py),(fx,fy)) Puntos de inicio y fin del segmento 1
#        seg2: ((px,py),(fx,fy)) Puntos de inicio y fin del segmento 1
#Esta funcion devuelve la distancia entre el segmento 1 y el 2
#Return segments_distance() Funcion que calcula la distancia entre dos segmentos
def segmentsDistance(seg1, seg2):
    return segments_distance(seg1[0][0], seg1[0][1], seg1[1][0], seg1[1][1],
                             seg2[0][0], seg2[0][1], seg2[1][0], seg2[1][1])
#Entrada:
#        x11: Segmento 1 Punto 1 coordedana x
#        y11: Segmento 1 Punto 1 coordedana y
#        x12: Segmento 1 Punto 2 coordedana x
#        y12: Segmento 1 Punto 2 coordedana y 
#        x21: Segmento 2 Punto 1 coordedana x
#        y21: Segmento 2 Punto 1 coordedana y
#        x22: Segmento 2 Punto 2 coordedana x
#        y22: Segmento 2 Punto 2 coordedana y
#Funcion que calcula la distancia entre dos segmentos que no se cruzan 
#RETURN: min(distances) la distancia minima de todas las posibles distancias
def segments_distance(x11, y11, x12, y12, x21, y21, x22, y22):
    """distance between two segments in the plane:
    one segment is (x11, y11) to (x12, y12)
    the other is   (x21, y21) to (x22, y22)
    """
    if segments_intersect(x11, y11, x12, y12, x21, y21, x22, y22): 
            return 0
    # try each of the 4 vertices w/the other segment
    distances = []
    distances.append(point_segment_distance(x11, y11, x21, y21, x22, y22))
    distances.append(point_segment_distance(x12, y12, x21, y21, x22, y22))
    distances.append(point_segment_distance(x21, y21, x11, y11, x12, y12))
    distances.append(point_segment_distance(x22, y22, x11, y11, x12, y12))
    return min(distances)
#Entrada:
#        x11: Segmento 1 Punto 1 coordedana x
#        y11: Segmento 1 Punto 1 coordedana y
#        x12: Segmento 1 Punto 2 coordedana x
#        y12: Segmento 1 Punto 2 coordedana y 
#        x21: Segmento 2 Punto 1 coordedana x
#        y21: Segmento 2 Punto 1 coordedana y
#        x22: Segmento 2 Punto 2 coordedana x
#        y22: Segmento 2 Punto 2 coordedana y
#Funcion boleana que calcula si dos segmentos se cruzan o no
#RETURN: true/false si se cruzan o no 
def segments_intersect(x11, y11, x12, y12, x21, y21, x22, y22):
    """ whether two segments in the plane intersect:
      one segment is (x11, y11) to (x12, y12)
      the other is   (x21, y21) to (x22, y22)
    """
    dx1 = x12 - x11
    dy1 = y12 - y11
    dx2 = x22 - x21
    dy2 = y22 - y21
    delta = dx2 * dy1 - dy2 * dx1
    if delta == 0: 
        return False  # parallel segments
    s = (dx1 * (y21 - y11) + dy1 * (x11 - x21)) / delta
    t = (dx2 * (y11 - y21) + dy2 * (x21 - x11)) / (-delta)
    return (0 <= s <= 1) and (0 <= t <= 1)
#Entrada:
#        px: Punto coordenada x 
#        py: Punto coordenada y
#        x1: Punto 1 coordenada x 
#        y1: Punto 1 coordenada y
#        x2: Punto 2 coordenada x 
#        y2: Punto 2 coordenada y
#Funcion que calcula la distancia desde un punto dado a un segmento.
#RETURN: DIstancia desde el punto al segmento
def point_segment_distance(px, py, x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    if dx == dy == 0:  # the segment's just a point
        return math.hypot(px - x1, py - y1)

    # Calculate the t that minimizes the distance.
    t = ((px - x1) * dx + (py - y1) * dy) / (dx * dx + dy * dy)

    # See if this represents one of the segment's
    # end points or a point in the middle.
    if t < 0:
        dx = px - x1
        dy = py - y1
    elif t > 1:
        dx = px - x2
        dy = py - y2
    else:
        near_x = x1 + t * dx
        near_y = y1 + t * dy
        dx = px - near_x
        dy = py - near_y

    return math.hypot(dx, dy)
#Entrada:
#        vA: Punto A
#        vB: Punto B
#Funcion que devuelve la multiplicacion de las coordenadas x de los dos puntos 
#mas la multiplicacion de las coordenadas y de los dos puntos
#Return: la multiplicacion de las coordenadas x de los dos puntos 
#        mas la multiplicacion de las coordenadas y de los dos puntos
def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]
#Entrada:
#        lineA:linea o segmento A
#        lineB:linea o segmento A
#Funcion que dadas dos rectas calcula el angulo que forman entre ellas.
#Return ang_deg: Devuelve el angulo que forman dichas rectas en grados.
def ang(lineA, lineB):
    # Get nicer vector form
    vA = [(lineA[0][0]-lineA[1][0]), (lineA[0][1]-lineA[1][1])]
    vB = [(lineB[0][0]-lineB[1][0]), (lineB[0][1]-lineB[1][1])]
    # Get dot prod
    dot_prod = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_prod/magA/magB
    if cos_ > 1: cos_ =1
    # Get angle in radians and then convert to degrees
    angle = math.acos(cos_)
    # Basically doing angle <- angle mod 360
    ang_deg = math.degrees(angle)%360

    if ang_deg-180>=0:
        # As in if statement
        return 360 - ang_deg
    else: 
        return ang_deg
#Entrada:
#        epsilon1: Distancia que si superan dos rectas no une
#        epsilon2: Angulo que si superan dos rectas no une
#        lines: Lineas producidas al aplicar la transformada
#        G: Grafo donde añadir los nodos que tengamso que unir
#Funcion que combina las lineas cercanas producidas por la Transformada probabilistica
#de hough que cumplen ciertos parametros y las añade a un grafo. 
#RETURN: devolvemos el Grafo completo donde estan los nodos que hemso fusionado.
def combina(epsilon1,epsilon2,lines,G): 
    for i in range(len(lines)):
        G.add_node(i)

    for i in range(len(lines)-1):
        for j in range(i+1,len(lines)):
            distance = segmentsDistance(lines[i],lines[j])
            #print(angle)
            if distance <= epsilon1 :
                angle = ang(lines[i],lines[j])
                #print(distance)
                if angle <= epsilon2:
                    #print("combina ",i,j)
                    G.add_edge(i,j) 
    return G

#Entrada.
#        p1: Punto 1
#        p2: Punto 2
#Calculamos la distancia entre dos puntos.
#Return distancia entre los puntos.
def pointDistance(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    return (dx*dx + dy*dy)**0.5

#Entrada: segmentosList lista con todos los segmentos a combinar
#Combinamos los segmentos
#Lista con los segmentos combinados que calculamos con la teoria de grafos.
def combinaSegmentos(segmentosList):
    #print("combina",segmentosList)
    Xs=list(map(lambda x:[x[0][0],x[1][0]],segmentosList))
    Ys=list(map(lambda x:[x[0][1],x[1][1]],segmentosList))
    xMax=np.max(Xs)
    yMax=np.max(Ys)
    xMin=np.min(Xs)
    yMin=np.min(Ys)
    if (xMax,yMax) in set(map(lambda x:x[0],segmentosList)):
        #print("devuelvo",((xMax,yMax),(xMin,yMin)))
        return ((xMax,yMax),(xMin,yMin))
    else:
        #print("devuelvo",((xMax,yMin),(xMin,yMax)))
        return ((xMax,yMin),(xMin,yMax))
#Entrada.
#        k_components K_componentes del grafo en el que decimos que rectas tenemos que unir
#        lines lineas calculadas por la tranformada de hough
#Funcion que dadas las lineas calculadas y las que nos marca el grafo que tenemso que unir
#devuelve lso segmentos que de verdad ha detectado el algoritmo
def segmentosVerdad(k_components,lines):
    segmentosDeVerdad = []
    for i in range(len(k_components[1])):
        segmentos=list(map(lambda x:lines[x],k_components[1][i]))
        segmentosDeVerdad.append(combinaSegmentos(segmentos))
    return segmentosDeVerdad
#Entrada:
#        imagen: Imagen qeu queremso mostrar.
#        segmentos: segmentos que vamso a mostrar dentro de la imagen.
#Procedimiento que usaremos para msoitrar los datos calculados pero solo es apra pruebas
#esta funcion no sera transladada cuando necesitemos la interfaz(SOLO NOTEBOOK)
#Return: No tiene
def mostrar(imagen,segmentos):
    plt.figure(figsize=(20, 20)) 
    plt.imshow(imagen, cmap=plt.cm.gray)
    for line in segmentos:
        p0, p1 = line
        plt.plot((p0[0], p1[0]), (p0[1], p1[1]),'b')
    #fig.savefig('foo.jpg',bbox_inches='tight')
    print(len(segmentos),"hay 15 lineas realmente")

In [2]:
#Saber version de python
#import sys
#print(sys.version_info)


sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


In [7]:
import sys
from PyQt4 import QtGui
from PyQt4 import QtCore
from PyQt4.QtCore import *
from PyQt4.QtGui  import *

from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt4agg import NavigationToolbar2QT as NavigationToolbar
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import random
import numpy as np
##Clase Ventana 
#Esta clase contendra la ventana principal de la aplcacion en la que colgaran los diferentes 
#componentes.
class MyMainWindow(QtGui.QMainWindow):

    def __init__(self, parent=None):

        super(MyMainWindow, self).__init__(parent)
        self.resize(900, 700)

        #fname = QFileDialog.getOpenFileName(self, 'Open file', 
        # 'c:\\',"Image files (*.jpg *.gif)")
        openFile = QtGui.QAction("&Abrir imagen", self)
        openFile.setShortcut("Ctrl+O")
        openFile.setStatusTip('Abrir imagen')
        openFile.triggered.connect(self.file_open)     

        self.statusBar()

        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('&Cargar')
        fileMenu.addAction(openFile)
        self.styleChoice = QtGui.QLabel("Cargar imagen para iniciar", self)
        self.styleChoice.setAlignment(QtCore.Qt.AlignCenter)

        laoutPrincipal = QtGui.QHBoxLayout()
        laoutPrincipal.addWidget(self.styleChoice)
        #self.styleChoice.move(10,100)
        self.styleChoice.setStyleSheet('color: red')
        #self.styleChoice.setGeometry(30, 100, 600, 50)
        font = QtGui.QFont("Times",35,QtGui.QFont.Bold,True)
        self.styleChoice.setFont(font)
        #self.setLayout(laoutPrincipal)#Asignamos como principal el principal
        central_widget = QtGui.QWidget()
        central_widget.setLayout(laoutPrincipal)
        self.setCentralWidget(central_widget)
    def file_open(self):
        self.path = QFileDialog.getOpenFileName(self, 'Open file', 
         'c:\\',"Image files (*.jpg *.gif)")
        #self.styleChoice.close() 
        self.form_widget = Window(self.path,self) 
        self.setCentralWidget(self.form_widget)
       
        self.path,
class Window(QtGui.QWidget):
    def __init__(self,path, parent=None ):
        super(Window, self).__init__(parent)
        self.resize(900, 700)
        #Figura sobre la que trabajaremos
        self.fig = Figure((6.5, 5.0), tight_layout=True)
        self.ax = self.fig.add_subplot(111)
        #---------Guardar Coordenadas--------------
        self.c1=0
        self.c2=0
        self.tamSegmenVerdad=-1
        self.lineas=[]
        self.pintados=[]
        self.rowActual=-1
        self.SelecAnte=None        
        #---------------------------------------------------------------
        
        #---------MENU cargar imagen------------------------------------
        self.path=path
       
        #---------------------------------------------------------------
        
        #------Para mostrar la Imagen del inicio--------------------------
        self.img=leerImagen(path)
        self.distance_red=distanciaAlRojo(self.img)
        self.imgBin=binarizar(self.distance_red)
        #self.imgBinCrop,self.imgCrop=cropImg(self.imgBin,self.img)
        self.ax = self.fig.add_subplot(111)
        self.ax.set_title('Figura sin lineas')
        self.ax.set_xlim([0,self.img.shape[1]])
        self.ax.set_ylim([0,self.img.shape[0]])
        self.ax.imshow(self.img,cmap=plt.cm.gray,origin='lower',vmax=1, interpolation='nearest')
    #-------------------------------------------------------
        
        #------Creamos los componentes -------------------------
        self.canvas = FigureCanvas(self.fig)
        self.toolbar = NavigationToolbar(self.canvas, self)

        self.button = QtGui.QPushButton('Calcular Lineas')
        self.button.clicked.connect(self.plot1)
        
        self.button2 = QtGui.QPushButton('Corregir Lineas')
        self.button2.clicked.connect(self.plot2)
        
        self.button3 = QtGui.QPushButton('Añadir punto')
        self.button3.clicked.connect(self.plot3)
        self.button3.setEnabled(False)

        self.button4 = QtGui.QPushButton('Añadir segmentos')
        self.button4.clicked.connect(self.anadirPuntos)  
        self.button4.setEnabled(False)

        self.button5 = QtGui.QPushButton('Borrar seleccionado')
        self.button5.clicked.connect(self.borrarSelec)
        
        #self.button6 = QtGui.QPushButton('Mostrar Tabla')
        #self.button6.clicked.connect(self.mostrarTabla)
        
        self.button7 = QtGui.QPushButton('Guardar tabla')
        self.button7.clicked.connect(self.guardarTabla)
                
        self.button8 = QtGui.QPushButton('Limpiar tabla')
        self.button8.clicked.connect(self.limpiarTabla)
        
        self.P1=QtGui.QLabel("P_1:")
        self.P2=QtGui.QLabel("P_2:")
        
        self.P1_x=QtGui.QLabel("0")
        self.P1_y=QtGui.QLabel("0")
        
        self.P2_x=QtGui.QLabel("0")
        self.P2_y=QtGui.QLabel("0")
        
        tabs= QtGui.QTabWidget()
        
        tab1 = QtGui.QWidget()
        tab2 = QtGui.QWidget()
        tab3 = QtGui.QWidget()
        
        tabs.addTab(tab1,"Lineas pintadas")
        tabs.addTab(tab2,"Corregir lineas")
        tabs.addTab(tab3,"Automático")
        
        self.table = QtGui.QTableWidget(self)
        self.table.setRowCount(0)
        self.table.setColumnCount(4)
        self.table.setEditTriggers(QtGui.QAbstractItemView.NoEditTriggers)
        self.table.itemSelectionChanged.connect(self.selected_row)

        self.header = self.table.horizontalHeader()
        self.header.setResizeMode(QHeaderView.Stretch)
        self.table.setHorizontalHeaderLabels(['P1X', 'P1Y', 'P2X', 'P2Y'])

        
        #-------------------------------------------------------

        #------Asignamos los componentes a un layout----------
        layoutTabs=QtGui.QHBoxLayout()
        layoutTab1=QtGui.QHBoxLayout()
        layoutTab2=QtGui.QHBoxLayout()
        layoutTab3=QtGui.QHBoxLayout()
        
        #layout principal - - 
        
        laoutPrincipal = QtGui.QHBoxLayout()
        # Layout (-) - /n 1
        layoutPrimero = QtGui.QVBoxLayout() 
        layoutPrimero.addWidget(self.toolbar)
        layoutPrimero.addWidget(self.canvas)
        # Layout - (-) /n 1 y 2
        layoutSegundo = QtGui.QVBoxLayout()      
        layoutSegundo.addWidget(self.button)
        layoutSegundo.setAlignment(QtCore.Qt.AlignTop)

        layoutPestaña1 = QtGui.QVBoxLayout()
        layoutPestaña1.addWidget(self.button2)


        
        #layout - (-) /n 3 y 4
        layautCorregirPunto1 = QtGui.QVBoxLayout()
        layautCorregirPunto1.setAlignment(QtCore.Qt.AlignTop)
        layautCorregirPunto1.addWidget(self.P1)
        layoutPunto1 = QtGui.QHBoxLayout()
        layoutPunto1.addWidget(self.P1_x)
        layoutPunto1.addWidget(self.P1_y)
        layoutPunto2 = QtGui.QHBoxLayout()
        layoutPunto2.addWidget(self.P2_x)
        layoutPunto2.addWidget(self.P2_y)
        layautCorregirPunto1.addLayout(layoutPunto1)
        layautCorregirPunto1.addWidget(self.P2)
        layautCorregirPunto1.addLayout(layoutPunto2)
        #Añadimos al layout - (-)
        layoutPestaña1.addLayout(layautCorregirPunto1)
        layoutPestaña1.addWidget(self.button3)
        layoutPestaña1.addWidget(self.button5)
        layoutPestaña1.addWidget(self.button8)
        layoutPestaña1.addWidget(self.table)
        layoutPestaña1.addWidget(self.button4)
        #layoutPestaña1.addWidget(self.button6)
        layoutPestaña1.addWidget(self.button7)
        
        #Añadimos las dos col al layaut principal
        
        layoutTab1.addLayout(layoutSegundo)
        
        layoutTab2.addLayout(layoutPestaña1)
        
        layoutTab3.addLayout(layoutSegundo)
        
        tab1.setLayout(layoutTab1)
        tab2.setLayout(layoutTab2)
        tab3.setLayout(layoutTab3)
        
        layoutTabs.addWidget(tabs)
        layoutTabs.setAlignment(QtCore.Qt.AlignRight)

        laoutPrincipal.addLayout(layoutPrimero)
        laoutPrincipal.setStretch(0,2)
        laoutPrincipal.addLayout(layoutTabs)
        laoutPrincipal.setStretch(2,3)

        #laoutPrincipal.addWidget(tabs)
        #laoutPrincipal.addLayout(layoutSegundo)
        
        self.setLayout(laoutPrincipal)#Asignamos como principal el principal
        #-------------------------------------------------------------------
                     
    def selected_row(self):
        self.rowActual=self.table.currentRow()
        row=self.rowActual
        if row>=0:
            p1x = self.table.item(row,0)
            p1x=p1x.text()
            p1y = self.table.item(row,1)
            p1y=p1y.text()
            p2x = self.table.item(row,2)
            p2x=p2x.text()
            p2y = self.table.item(row,3)
            p2y=p2y.text()

            #self.ax.lines[0].remove()
            self.ax.hold(True) 
            self.ax.set_title('Figura con lineas')
            #self.ax.set_xlim([0,self.img.shape[1]])
            #self.ax.set_ylim([0,self.img.shape[0]])
            sel,=self.ax.plot((np.int32(p1x), np.int32(p2x)), (np.int32(p1y) , np.int32(p2y)),'yellow',linewidth=2.0)
            if self.SelecAnte!=None:            
                self.ax.lines.remove(self.SelecAnte)
            self.SelecAnte=sel
            self.ax.hold(False)
            self.canvas.draw()
        #print(self.tamSegmenVerdad)
        #if self.tamSegmenVerdad!=-1:
        #    self.ax.lines[self.tamSegmenVerdad].remove()
        
        #print(self.table.currentRow())
    
    def plot1(self):        
        sinRuido=reducirGrosor(self.imgBin)
        lines=proHough(10,5,11,sinRuido)        
        G=nx.Graph()
        G=combina(8,4,lines,G)
        k_components = apxa.k_components(G)
        segmentosDeVerdad=segmentosVerdad(k_components,lines)
        #print(sys.version_info)
        self.pintarImagenYsegmentos(segmentosDeVerdad)
        self.canvas.draw()
        self.lineas=segmentosDeVerdad
        self.tamSegmenVerdad=len(self.lineas)
        self.button4.setEnabled(True)

        
    def pintarImagenYsegmentos(self,segmentos):
        self.ax = self.fig.add_subplot(111)
        self.ax.set_title('Figura con lineas')
        self.ax.set_xlim([0,self.img.shape[1]])
        self.ax.set_ylim([0,self.img.shape[0]])
        self.ax.imshow(self.img,cmap=plt.cm.gray,origin='lower',vmax=1, interpolation='nearest')
        self.ax.hold(True)
        for line in segmentos:
            p0, p1 = line
            self.ax.set_title('Figura con lineas')
            self.ax.set_xlim([0,self.img.shape[1]])
            self.ax.set_ylim([0,self.img.shape[0]])            
            self.ax.plot((p0[0], p1[0]), (p0[1], p1[1]),'b',linewidth=2.0)
            #-print(type(p0[0]))
        self.ax.hold(False)
        self.canvas.draw()

        
    def plot2(self):
        self.button3.setEnabled(False)
        self.P2.setStyleSheet('color: black')
        self.P1.setStyleSheet('color: black')

        self.P1_x.setText("0")
        self.P1_y.setText("0")
        self.P2_x.setText("0")
        self.P2_y.setText("0")        
        self.button2.setEnabled(False)
        coords = []     
        def onclick(event):
            ix, iy = event.xdata, event.ydata
            if ix != None and iy != None:
                if len(coords) == 0:
                    self.P1.setStyleSheet('color: Red')
                    self.P1_x.setText(str(round(ix,0)))
                    self.P1_y.setText(str(round(iy,0)))
                    #print ('x = %d, y = %d'%(ix, iy))
                    coords.append((ix, iy))
                    self.c1=(ix,iy)

                else:
                    self.P2.setStyleSheet('color: Red')
                    self.P2_x.setText(str(round(ix,0)))
                    self.P2_y.setText(str(round(iy,0)))
                    self.c2=(ix,iy)                    
                    #print (' = %d, y = %d'%(ix, iy))
                    coords.append((ix, iy))
                    self.fig.canvas.mpl_disconnect(cid)
                    self.button2.setEnabled(True)
                    self.button3.setEnabled(True)
                    

            return coords
        cid = self.fig.canvas.mpl_connect('button_press_event', onclick)
        self.canvas.draw()

    def plot3(self):
        self.P2.setStyleSheet('color: black')
        self.P1.setStyleSheet('color: black')

        self.P1_x.setText("0")
        self.P1_y.setText("0")
        self.P2_x.setText("0")
        self.P2_y.setText("0")
        if self.c1!=None and self.c2!=None:        
            row = self.table.rowCount()
            self.table.insertRow(row)
            self.table.setItem(row, 0, QtGui.QTableWidgetItem(str(int(self.c1[0]))))
            self.table.setItem(row, 1, QtGui.QTableWidgetItem(str(int(self.c1[1]))))
            self.table.setItem(row, 2, QtGui.QTableWidgetItem(str(int(self.c2[0]))))
            self.table.setItem(row, 3, QtGui.QTableWidgetItem(str(int(self.c2[1]))))
            #print(self.c2)
            self.pintados.append((self.c1,self.c2))
            self.tamSegmenVerdad=len(self.lineas)
            #print(self.lineas)
            self.c1=None
            self.c2=None
        self.mostrarTabla()
        self.button3.setEnabled(False)

    def anadirPuntos(self):
        self.limpiarTabla()
        row = self.table.rowCount()
        if len(self.lineas)!=0:
            for i in self.lineas:
                self.table.insertRow(row)
                self.table.setItem(row, 0, QtGui.QTableWidgetItem(str(int(i[0][0]))))
                self.table.setItem(row, 1, QtGui.QTableWidgetItem(str(int(i[0][1]))))
                self.table.setItem(row, 2, QtGui.QTableWidgetItem(str(int(i[1][0]))))
                self.table.setItem(row, 3, QtGui.QTableWidgetItem(str(int(i[1][1]))))
                row+=1
            self.button4.setEnabled(False)
            self.mostrarTabla()
                
    def borrarSelec(self):
        if self.rowActual!=-1:
            self.table.removeRow(self.rowActual)            
            self.rowActual=-1
            self.mostrarTabla()

    def mostrarTabla(self):
        row = self.table.rowCount()
        segmentos=[]
        x1,x2,y1,y2=0,0,0,0
        for i in range(row):            
            x1=int(self.table.item(i,0).text())
            x2=int(self.table.item(i,1).text())
            y1=int(self.table.item(i,2).text())
            y2=int(self.table.item(i,3).text())
            #print(x1,x2,y1,y2)
            segmentos.append(((x1,x2),(y1,y2)))
            
        self.pintarImagenYsegmentos(segmentos)
        self.SelecAnte=None
        self.canvas.draw()

        #pintarImagenYsegmentos(,1)
    def guardarTabla(self):
        print("oko")
    def limpiarTabla(self):
        for i in reversed(range(self.table.rowCount())):
            self.table.removeRow(i)
        self.mostrarTabla()

if __name__ == '__main__':
    app = QtGui.QApplication(sys.argv)
    main = MyMainWindow()
    main.show()
    sys.exit(app.exec_())

SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.
